# Fine-tuning Whisper for Vietnamese ASR
This notebook demonstrates how to fine-tune OpenAI's Whisper model on a Vietnamese speech dataset using Hugging Face Transformers. The workflow includes environment setup, data loading, preprocessing, model training, and evaluation.

## 1. Environment Setup
Install the required libraries: `transformers`, `datasets`, `torchaudio`, and `jiwer` for evaluation.

In [2]:
!pip install transformers datasets torchaudio jiwer --quiet

In [3]:
import os
import pandas as pd
from datasets import load_dataset, Dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, TrainingArguments, Trainer
import torch

/home/haipham2407/miniconda3/envs/nlp_prj/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Load and Prepare Data
Assume your CSV files (`fpt_train.csv`, `fpt_val.csv`, `fpt_test.csv`) are in the current directory and contain columns: `path` (audio file path) and `transcription` (text).

In [4]:
def load_csv_to_dataset(csv_path):
    df = pd.read_csv(csv_path)
    ds = Dataset.from_pandas(df)
    ds = ds.cast_column("path", Audio(sampling_rate=16000))
    return ds

train_dataset = load_csv_to_dataset("fpt_train.csv")
val_dataset = load_csv_to_dataset("fpt_val.csv")
test_dataset = load_csv_to_dataset("fpt_test.csv")

## 3. Load Whisper Model and Processor

In [5]:
model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="vi", task="transcribe")

## 4. Preprocessing Function
Tokenize transcriptions and prepare input features.

In [ ]:
def prepare_dataset(batch):
    audio = batch["path"]
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
    return batch

# Use batched processing and avoid keeping everything in memory
train_dataset = train_dataset.map(
    prepare_dataset,
    batched=True,
    batch_size=8,  # or another small number
    num_proc=1,    # increase if you have more CPU cores
    load_from_cache_file=True,
    keep_in_memory=False
)
val_dataset = val_dataset.map(
    prepare_dataset,
    batched=True,
    batch_size=8,
    num_proc=1,
    load_from_cache_file=True,
    keep_in_memory=False
)

Map:   0%|          | 0/20735 [00:00<?, ? examples/s]



TypeError: list indices must be integers or slices, not str

## 5. Training Arguments and Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="./whisper-vi-finetuned",
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    warmup_steps=500,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
)

def compute_metrics(pred):
    from jiwer import wer
    pred_str = processor.batch_decode(pred.predictions, skip_special_tokens=True)
    label_str = processor.batch_decode(pred.label_ids, skip_special_tokens=True)
    return {"wer": wer(label_str, pred_str)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
)

## 6. Start Training

In [ ]:
trainer.train()

## 7. Evaluate on Test Set

In [ ]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

In [ ]:
# Save the fine-tuned model and processor for later use
model.save_pretrained("./whisper-vi-finetuned")
processor.save_pretrained("./whisper-vi-finetuned")
print("Model and processor saved to ./whisper-vi-finetuned")

---
This notebook provides a basic pipeline for fine-tuning Whisper on Vietnamese ASR data. You can further customize preprocessing, augmentation, and hyperparameters as needed.